In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
import sys
from pathlib import Path

In [ ]:
# ---------- USER CONFIG ----------
INPUT_SHP ="D:/2_Analytics/9_LULC_classification/automation/WF/WF/Settlement.shp"  # <-- change this to your shapefile path
BUFFER_METERS = 20               # buffer distance in meters
OUT_BUFFERED = "D:/2_Analytics/9_LULC_classification/automation/output/buffered_20m.shp"
OUT_DISSOLVED = "D:/2_Analytics/9_LULC_classification/automation/output/dissolved_buffered_20m.shp"
# ---------------------------------

def main():
    shp = Path(INPUT_SHP)
    if not shp.exists():
        print(f"ERROR: input file not found: {shp}")
        sys.exit(1)

    # Read shapefile
    gdf = gpd.read_file(str(shp))
    print("Loaded:", shp)
    print("Original CRS:", gdf.crs)

    if gdf.crs is None:
        print("ERROR: input shapefile has no CRS. You must provide a CRS for correct buffering.")
        print("If coordinates are in latitude/longitude you should set gdf.crs = 'EPSG:4326' and re-run.")
        sys.exit(1)

    # If CRS is geographic (degrees), reproject to a metric CRS
    try:
        is_geographic = gdf.crs.is_geographic
    except Exception:
        # fallback if .is_geographic not available
        is_geographic = ("degree" in str(gdf.crs).lower()) or ("4326" in str(gdf.crs))

    if is_geographic:
        # estimate a suitable UTM (local) projected CRS for minimal distortion
        try:
            utm_crs = gdf.estimate_utm_crs()  # geopandas >= 0.10
            print("Estimated UTM CRS:", utm_crs)
        except Exception:
            # fallback to Web Mercator (meters) if estimate_utm_crs not available
            utm_crs = "EPSG:3857"
            print("Could not estimate UTM CRS. Falling back to:", utm_crs)

        gdf_proj = gdf.to_crs(utm_crs)
        print("Reprojected to metric CRS for buffering.")
    else:
        gdf_proj = gdf.copy()
        print("CRS already projected (assumed metric).")

    # Make sure geometry is valid
    gdf_proj['geometry'] = gdf_proj['geometry'].buffer(0)  # fixes minor invalidities
    invalid = gdf_proj[~gdf_proj.is_valid]
    if not invalid.empty:
        print(f"Warning: {len(invalid)} invalid geometries remain (attempting buffer(0) didn't fix).")

    # Create 20 m buffer per feature
    buffered_geom = gdf_proj.geometry.buffer(BUFFER_METERS)
    buffered_gdf = gdf_proj.copy()
    buffered_gdf['geometry'] = buffered_geom
    print(f"Buffered each feature by {BUFFER_METERS} m.")

    # Dissolve/merge all buffered features into a single geometry
    # Option 1: dissolve by a dummy field
    dissolved = buffered_gdf.dissolve()  # returns one-row GeoDataFrame with merged geometry
    # Option 2 (alternative): unary_union = buffered_gdf.geometry.unary_union
    # combined_gdf = gpd.GeoDataFrame(geometry=[unary_union], crs=buffered_gdf.crs)
    print("Dissolved/merged buffered geometries into one geometry.")

    # Save outputs (in the same CRS used for buffering)
    buffered_gdf.to_file(OUT_BUFFERED)
    print("Saved per-feature buffered shapefile:", OUT_BUFFERED)

    dissolved.to_file(OUT_DISSOLVED)
    print("Saved dissolved buffered shapefile:", OUT_DISSOLVED)

    # Quick plot to visual check (original, buffered, dissolved)
    fig, axs = plt.subplots(1, 3, figsize=(15, 5))
    gdf_proj.plot(ax=axs[0], edgecolor='black', facecolor='none')
    axs[0].set_title("Original (projected)")

    buffered_gdf.plot(ax=axs[1], color='lightblue', edgecolor='blue')
    axs[1].set_title(f"Buffered ({BUFFER_METERS} m)")

    dissolved.plot(ax=axs[2], color='salmon', edgecolor='red')
    axs[2].set_title("Dissolved (merged)")

    for ax in axs:
        ax.set_axis_off()
    plt.tight_layout()
    plt.show()


if __name__ == "__main__":
    main()
